In [ ]:
"""from https://github.com/HeWeMel/nographs/discussions/12, Computing strongly connected components"""

In [ ]:
import operator # import lt, gt
from functools import update_wrapper, cmp_to_key
from itertools import chain, repeat
from collections import defaultdict
from decorator import decorator
import nographs as nog
from graphviz import Digraph

In [ ]:
dict_1= {0:{111,2,4}, 111:{3,5}, 2:{3,6,0}, 3:{37,0}, 37:{7}, 4:{5,6}, 5:{57}, 57:{7},
         6:{67}, 67:{7}, 7:{6} }

In [ ]:
class Adapter_Graphviz:
    """translation to GraphViz visitor"""
    def __init__(self, to_deco):
        update_wrapper(self, to_deco )
        self.dot = Digraph()
    def __call__(self, vert, _trav):
        """visitor wrapper"""
        #print(f' Adapter_Graphviz {vert=}')
        #print(f' Adapter_Graphviz {dict(_trav.paths._predecessor)}')
        self.dot.node( str(vert), str(vert) )
        
        pred = _trav.paths._predecessor[ vert ]
        if pred is not None:
            self.dot.edge( str( pred ), str( vert ), color="red" )

        for result, label in self.__wrapped__( (pred, vert), _trav):
            self.dot.edge( str(vert), str(result), str(label), )
            yield result, label

# def adapter_graphviz( wrapped ):
#     """translation to GraphViz visitor"""
#     dot = Digraph( node_attr = node_style, graph_attr = gr_style, )
#     @wraps( wrapped )
#     def wrapper_bg( _edge, _trav ):
#         dot.node( str(_edge), str(_edge) )
#         for result in wrapped(_edge, _trav):
#             dot.edge( str(_edge), str(result[0]), str(result[1]), )
#             yield result
#     wrapper_bg.dot = dot
#     return wrapper_bg

In [ ]:
class Components_Kosaraju_PhaseI:
    # def _lt_postorder( self, v1, v2 ):
    #     return operator.lt(self.visit_order[v1], self.visit_order[v2], )
    def __init__(self, wrapped):
        update_wrapper(self, wrapped )
        self.visit_order ={}
        # self.notify_parent ={}
        self.time =0
        # self.lt_postorder = self._lt_postorder # to be lifted with `update_wrapper`-s
    def __call__(self, edge, _trav):
        print(f'{edge=}')
        pred, vert = edge
        # print(f"{x=:4}   {self.time:2}")
        
        # self.visit_order[                     x ] = self.time
        for result in self.__wrapped__(vert, _trav):
            self.time += 1
            #print(f'{result=}')
            yield result[1],self.time
        self.visit_order[ pred ] = self.time
    def __repr__(self):
        return self.visit_order

In [ ]:
@decorator
def edge_name_as_target( wrapped, vert, _trav ):
    for item in wrapped( vert, _trav ):
        print(f'{item=}')
        yield (item, item)

#@edge_name_as_target
@Adapter_Graphviz
@Components_Kosaraju_PhaseI
def forward_I( vert, _):
    """wrapping the dict as `next_vertices` visitor function"""
    #print(f'{vert=}')
    return zip( repeat(vert), dict_1[ vert ])

In [ ]:
trav_forward = nog.TraversalBreadthFirst(next_labeled_edges=forward_I)
trav_forward.start_from( 0, build_paths=True )
list(trav_forward)

In [ ]:
forward_I.dot              # pylint: disable=pointless-statement
#[0, 2, 67, 3, 5, 6, 7, 37, 4, 111, 57]

In [ ]:
print(trav_forward.paths._predecessor_collection)
print(trav_forward.paths._predecessor)
print(trav_forward.visited)
forward_I.visit_order

dict(sorted(forward_I.visit_order.items(), key=lambda item: item[1]))

In [ ]:
#dir(trav_forward.paths)

In [ ]:
def next_component( vert ):
    """`next_vertices` for component traversal"""
    best_top = best_bot = vert
    for pretender in dict_1[ vert ]:
        if pretender > best_top:
            best_top = pretender
        elif pretender > best_bot:
            best_bot = pretender
    return (best_top, best_bot)
    

In [ ]:
def get_best( min_or_max, vert ):
    """`next_vertices` for component traversal Kosaraju"""
    best = min_or_max( chain( (vert,), dict_1[ vert ]) )
    if vert == best:
        return vert
    return get_best( min_or_max, best )

In [ ]:
def get_component( vert ):
    """`next_vertices` for component traversal"""
    return (get_best( min, vert ), get_best( max, vert ), )

In [ ]:
def invert_from_dict( graph ):
    rev = defaultdict(lambda:set())
    for vert, dests in graph.items():
        for dest in dests:
            rev[ dest ].add(vert)
    return rev
#list(zip(dir(forward.__wrapped__),dir(forward),))
#forward.visit_order
invert_from_dict( dict_1 )

In [ ]:
sorted( invert_from_dict( dict_1 )[7], 
       key=cmp_to_key( forward_I.lt_postorder ) )

In [ ]:
class Components_Kosaraju_PhaseII:
    def lt_postorder( self, v1, v2 ):
        return operator.lt(forward_I.visit_order[v1], forward_I.visit_order[v2], )
    def __init__(self, wrapped):
        update_wrapper(self, wrapped )
    def __call__(self, x, t):
        for result in self.__wrapped__(x, t):
            yield result
    def __repr__(self):
        return self.visit_order

In [ ]:
subtrees = sorted( trav_forward.visited,
       key=cmp_to_key( forward_I.lt_postorder ) )

#@Components_Kosaraju_PhaseI
inv_graph = invert_from_dict(dict_1)
@Adapter_Graphviz
def forward_II( vert, _):
    """wrapping the dict as `next_vertices` visitor function"""
    if vert ==-1:
        return subtrees
    return inv_graph[ vert ]

subtrees

In [ ]:
trav_II = nog.TraversalDepthFirst( forward_II )
trav_II.start_from( -1, build_paths=True )
list(trav_II)

In [ ]:
forward_II.dot

In [ ]:
trav_forward.visited

In [ ]:
sorted( trav_forward.visited,
       key=cmp_to_key( forward_I.lt_postorder ) )

In [ ]:
forward_I.visit_order